# 0. Imports and installations

In [ ]:
# !pip install polars
# !pip install pyahocorasick
# !pip install httpx trafilatura

In [ ]:
# IMPORTS

import re
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile
from io import BytesIO
import polars as pl
import multiprocessing
from multiprocessing import Pool, cpu_count
from urllib.parse import urljoin
import concurrent.futures
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)
parquet_dir = os.path.join(data_dir, "filtered_parquets")
os.makedirs(parquet_dir, exist_ok=True)

# 1. Obtaining the data

To obtain the data we access the GDELT 1.0 Event Database -- a repository where they store a dataset (in csv format) for every day since January 1st, 1979 with news articles from the whole world. This is updated daily and each file contains tens of thousands of news articles (their urls). 

In the pipeline below, we access the archive url and download the files for the period going from January 1st, 2020 to June 6th, 2021. The reason for this is that our terrorism dataset source only contains data up to June 2021 and we limited our period to 18 months because of the scale of the GDELT Database.

In [ ]:
# GDELT events archive URL
base_url = "http://data.gdeltproject.org/events/"

# get the list of ZIP files
response = requests.get(base_url)
soup = BeautifulSoup(response.content, 'html.parser')

start_period = '202001'
end_period = '202106'

zip_links = [
    a['href']
    for a in soup.find_all('a', href=True)
    if a['href'].endswith('.zip') and start_period <= a['href'][:6] <= end_period
]

# download and extract each ZIP file
def download_and_extract(link):
    zip_url = urljoin(base_url, link)
    try:
        r = requests.get(zip_url, stream=True, timeout=30)
        r.raise_for_status()
        with BytesIO() as tmp_buffer:
            for chunk in r.iter_content(chunk_size=8192):
                if chunk:
                    tmp_buffer.write(chunk)
            tmp_buffer.seek(0)
            with ZipFile(tmp_buffer) as z:
                z.extractall(data_dir)
    except Exception as e:
        print(f'Failed: {zip_url} - {e}')

with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    list(tqdm(executor.map(download_and_extract, zip_links), total=len(zip_links)))

Below is our reference dictionary of the columns contained in the GDELT csvs, as they are unnamed in the files. We decide to keep the following:
* Month, Year - format YYYYMMM
* Year
* Event Code
* Quad Class
* Goldstein Scale
* Average Tone
* Actor 1 state code
* Actor 3 state code
* Country - to filter by 'US'
* State
* Date
* URL

In [ ]:
# column_dict = {
#     0: 'event_id',
#     1: 'SQLDATE',
#     2: 'MonthYear',
#     3: 'Year',
#     4: 'FractionDate',
#     5: 'Actor1Code',
#     6: 'Actor1Name',
#     7: 'Actor1CountryCode',
#     8: 'Actor1KnownGroupCode',
#     9: 'Actor1EthnicCode',
#     10: 'Actor1Religion1Code',
#     11: 'Actor1Religion2Code',
#     12: 'Actor1Type1Code',
#     13: 'Actor1Type2Code',
#     14: 'Actor1Type3Code',
#     15: 'Actor2Code',
#     16: 'Actor2Name',
#     17: 'Actor2CountryCode',
#     18: 'Actor2KnownGroupCode',
#     19: 'Actor2EthnicCode',
#     20: 'Actor2Religion1Code',
#     21: 'Actor2Religion2Code',
#     22: 'Actor2Type1Code',
#     23: 'Actor2Type2Code',
#     24: 'Actor2Type3Code',
#     25: 'IsRootEvent',
#     26: 'EventCode',
#     27: 'EventBaseCode',
#     28: 'EventRootCode',
#     29: 'QuadClass',
#     30: 'GoldsteinScale',
#     31: 'NumMentions',
#     32: 'NumSources',
#     33: 'NumArticles',
#     34: 'AvgTone',
#     35: 'Actor1Geo_Type',
#     36: 'Actor1Geo_FullName',
#     37: 'Actor1Geo_CountryCode',
#     38: 'Actor1Geo_ADM1Code',
#     39: 'Actor1Geo_Lat',
#     40: 'Actor1Geo_Long',
#     41: 'Actor1Geo_FeatureID',
#     42: 'Actor2Geo_Type',
#     43: 'Actor2Geo_FullName',
#     44: 'Actor2Geo_CountryCode',
#     45: 'Actor2Geo_ADM1Code',
#     46: 'Actor2Geo_Lat',
#     47: 'Actor2Geo_Long',
#     48: 'Actor2Geo_FeatureID',
#     49: 'ActionGeo_Type',
#     50: 'ActionGeo_FullName',
#     51: 'ActionGeo_CountryCode',
#     52: 'ActionGeo_ADM1Code',
#     53: 'ActionGeo_Lat',
#     54: 'ActionGeo_Long',
#     55: 'ActionGeo_FeatureID',
#     56: 'date',
#     57: 'url',
# }

Next, we process the csv one by one. We upload them, keep the coulmns of interest, filter the rows belonging to the US, and drop duplicates. We then save each file as a parquet, as it is a more efficient file format. Doing it file by file allows us to make the process more efficient as well as to keep separate files for each date.

In [ ]:
csv_files = [f for f in os.listdir(data_dir) if f.endswith('.CSV')]

# df_us = pd.DataFrame()

# COLUMNS TO LOAD
# indexes
columns_to_load = [2, 3, 26, 29, 30, 34, 38, 45, 51, 52, 56, 57]

# dictionary with parquet naming of the columns
column_dict = {
    'column_3': 'month_year',
    'column_4': 'year',
    'column_27': 'event_code',
    'column_30': 'quad_class',
    'column_31': 'goldstein_scale',
    'column_35': 'avg_tone',
    'column_39': 'actor1_statecode',
    'column_46': 'actor2_statecode',
    'column_52': 'country',
    'column_53': 'state',
    'column_57': 'date',
    'column_58': 'url',  
}


parquet_dir = os.path.join(data_dir, "filtered_parquets")
os.makedirs(parquet_dir, exist_ok=True)

# Iterate over CSV files, format the DF, and extract title from URL
for csv_file in tqdm(csv_files, desc="Processing CSV files"):
    file_path = os.path.join(data_dir, csv_file)

    df = pl.read_csv(
        file_path,
        separator='\t',
        columns=columns_to_load,
        has_header=False,
        null_values=['---', 'X', ''],  # handle potential null values
        schema_overrides={'column_27': pl.Utf8},  
        ignore_errors=False  
    ).rename(column_dict)

    # convert event_code to Int64 after cleaning
    df = df.with_columns(
        pl.col('event_code').str.replace_all(r'\D', '')  
        .cast(pl.Int64, strict=False)  
    )

    # keep only US + remove duplicates
    df_lazy = df.lazy() 
    del df
    df_us = df_lazy.filter(pl.col('country') == 'US').drop('country').unique().collect()

    parquet_path = os.path.join(parquet_dir, csv_file.replace(".CSV", ".parquet"))
    df_us.write_parquet(parquet_path)

    del df_us

print("All CSV files have been processed and saved as Parquet files.")

Processing CSV files: 100%|██████████| 547/547 [01:17<00:00,  7.06it/s]

All CSV files have been processed and saved as Parquet files.


Finally, we create a joint dataset with the whole time period.

In [2]:
parquet_files = [os.path.join(parquet_dir, f) for f in os.listdir(parquet_dir) if f.endswith('.parquet')]
df_all = pl.concat([pl.read_parquet(f) for f in parquet_files])

In [ ]:
df_all.head()

month_year,year,event_code,quad_class,goldstein_scale,avg_tone,actor1_statecode,actor2_statecode,state,date,url
i64,i64,i64,i64,f64,f64,str,str,str,i64,str
192001,1920,43,1,2.8,-3.924092,null,"""USVA""","""USVA""",20200101,"""https://www.rockymounttelegram…"
192001,1920,20,1,3.0,-1.827875,"""USCA""","""USCA""","""USCA""",20200101,"""https://www.sfchronicle.com/bu…"
192001,1920,10,1,0.0,0.0,"""USCA""","""USCA""","""USCA""",20200101,"""https://www.49ers.com/about-us…"
192001,1920,190,4,-10.0,-2.040816,"""USDC""","""USDC""","""USDC""",20200101,"""https://www.newsweek.com/giuli…"
192001,1920,173,4,-5.0,-8.445946,null,"""USCA""","""USCA""",20200101,"""https://www.dailyrepublic.com/…"


The scale of the dataset is way to big - it has over 18 million rows and 34,000 news articles per day. Due to time and resource constraints we cannot process this, so we pick 5 news per state per day. It is a small number but it is the largest size we could choose to be able to properly run the code below without major time delays.

In [3]:
print('----BEFORE FILTERING----')
print(f"Total rows after cleaning: {df_all.height:,}")
print('Average rows per day:', f"{round(df_all.height / len(parquet_files)):,}")

rows_per_state_per_day = 5
df_filtered = (
    df_all
    .group_by(["state", "date"], maintain_order=True)
    .head(rows_per_state_per_day)  
)

print('\n----AFTER FILTERING----')
print(f"Total rows after filtering: {df_filtered.height:,}")
print('Average rows per day:', f"{round(df_filtered.height / len(parquet_files)):,}")

----BEFORE FILTERING----
Total rows after cleaning: 18,792,604
Average rows per day: 34,356

----AFTER FILTERING----
Total rows after filtering: 145,030
Average rows per day: 265


## 1.2. Extracting article title and body

We extract the title and the body by scraping the url links in the GDELT dataset. This process is very long and not always successfull (text is extracted successfully in around 50% of the articles).

In [ ]:
import httpx
import trafilatura
import polars as pl
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm

# Set up URLs from your Polars DF
urls = df_filtered['url'].to_list()

# Fetch function
def fetch_article(url, timeout=15):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36"
    }
    try:
        with httpx.Client(headers=headers, follow_redirects=True, timeout=timeout) as client:
            response = client.get(url)
            if response.status_code == 200:
                html = response.text
                article_text = trafilatura.extract(html)
                metadata = trafilatura.extract_metadata(html)

                title = getattr(metadata, "title", None) if metadata else None
                if title is None and isinstance(metadata, dict):
                    title = metadata.get("title")

                if article_text:
                    return {"url": url, "title": title, "full_text": article_text}
    except Exception:
        pass
    return {"url": url, "title": None, "full_text": None}

# run concurrently and add results to a list
results = []

with ThreadPoolExecutor(max_workers=32) as executor:
    futures = {executor.submit(fetch_article, url): url for url in urls}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Fetching"):
        results.append(future.result())

# add to our dataframe
df_results = pl.DataFrame(results)

df_filtered = df_filtered.unique(subset=["url"])
df_results = df_results.unique(subset=["url"])

df_filtered = df_filtered.join(df_results, on="url", how="left")


In [22]:
successful_extractions = df_filtered.select(pl.col("full_text").n_unique()).item() - 1
print(f'Number of news articles texts correctly extracted: {successful_extractions:,}')

Number of news articles texts correctly extracted: 77,661


In [9]:
df_filtered.head()

state,date,month_year,year,event_code,quad_class,goldstein_scale,avg_tone,actor1_statecode,actor2_statecode,url,title,full_text
str,i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str
"""USMO""",20210514,202105,2021,16,1,-2.0,-8.934073,"""USMO""","""USMO""","""https://www.natlawreview.com/a…","""State of the Law for Business …","""It’s been a year since COVID-1…"
"""USMO""",20210514,202105,2021,141,3,-6.5,-0.808625,"""USMO""","""USMO""","""https://www.kcur.org/health/20…","""Medicaid Expansion Supporters …","""A day after Missouri Gov. Mike…"
"""USMO""",20210529,202105,2021,13,1,0.4,-6.008584,"""USMO""","""USMO""","""https://www.dailystar.co.uk/ne…","""Elderly woman sucker-punched t…","""Elderly woman sucker-punched t…"
"""USAR""",20200207,202002,2020,16,1,-2.0,-8.0,"""USAR""",null,"""https://www.houstonchronicle.c…",null,null
"""USNH""",20201206,202012,2020,70,2,7.0,0.088106,"""USNH""","""USNH""","""https://www.fosters.com/story/…","""Historically Speaking: Adventu…","""Historically Speaking: Adventu…"


# 2. Data preparation

## 2.1. Extracting States from the news text

We extract the states from the news text by finding mentions to the state name, the capital, or the second and third cities.

In [ ]:
# df_filtered.write_parquet("data/FILTERED_DATAFRAME.parquet")
# df_filtered = pl.read_parquet("data/FILTERED_DATAFRAME.parquet")

In [ ]:
states_dict = { # CONTAINS STATE NAME + CAPITAL + 2nd AND 3rd LARGEST CITIES
    'AL': ['Alabama', 'Montgomery', 'Birmingham', 'Mobile'],
    'AK': ['Alaska', 'Juneau', 'Anchorage', 'Fairbanks'],
    'AZ': ['Arizona', 'Phoenix', 'Tucson', 'Mesa'],
    'AR': ['Arkansas', 'Little Rock', 'Fort Smith', 'Fayetteville'],
    'CA': ['California', 'Sacramento', 'Los Angeles', 'San Diego'],
    'CO': ['Colorado', 'Denver', 'Colorado Springs', 'Aurora'],
    'CT': ['Connecticut', 'Hartford', 'Bridgeport', 'New Haven'],
    'DE': ['Delaware', 'Dover', 'Wilmington', 'Newark'],
    'FL': ['Florida', 'Tallahassee', 'Jacksonville', 'Miami'],
    'GA': ['Georgia', 'Atlanta', 'Augusta', 'Columbus'],
    'HI': ['Hawaii', 'Honolulu', 'Hilo', 'Kailua'],
    'ID': ['Idaho', 'Boise', 'Meridian', 'Nampa'],
    'IL': ['Illinois', 'Springfield', 'Chicago', 'Aurora'],
    'IN': ['Indiana', 'Indianapolis', 'Fort Wayne', 'Evansville'],
    'IA': ['Iowa', 'Des Moines', 'Cedar Rapids', 'Davenport'],
    'KS': ['Kansas', 'Topeka', 'Wichita', 'Overland Park'],
    'KY': ['Kentucky', 'Frankfort', 'Louisville', 'Lexington'],
    'LA': ['Louisiana', 'Baton Rouge', 'New Orleans', 'Shreveport'],
    'ME': ['Maine', 'Augusta', 'Portland', 'Lewiston'],
    'MD': ['Maryland', 'Annapolis', 'Baltimore', 'Frederick'],
    'MA': ['Massachusetts', 'Boston', 'Worcester', 'Springfield'],
    'MI': ['Michigan', 'Lansing', 'Detroit', 'Grand Rapids'],
    'MN': ['Minnesota', 'Saint Paul', 'Minneapolis', 'Rochester'],
    'MS': ['Mississippi', 'Jackson', 'Gulfport', 'Southaven'],
    'MO': ['Missouri', 'Jefferson City', 'Kansas City', 'St. Louis'],
    'MT': ['Montana', 'Helena', 'Billings', 'Missoula'],
    'NE': ['Nebraska', 'Lincoln', 'Omaha', 'Bellevue'],
    'NV': ['Nevada', 'Carson City', 'Las Vegas', 'Henderson'],
    'NH': ['New Hampshire', 'Concord', 'Manchester', 'Nashua'],
    'NJ': ['New Jersey', 'Trenton', 'Newark', 'Jersey City'],
    'NM': ['New Mexico', 'Santa Fe', 'Albuquerque', 'Las Cruces'],
    'NY': ['New York', 'Albany', 'New York City', 'Buffalo'],
    'NC': ['North Carolina', 'Raleigh', 'Charlotte', 'Greensboro'],
    'ND': ['North Dakota', 'Bismarck', 'Fargo', 'Grand Forks'],
    'OH': ['Ohio', 'Columbus', 'Cleveland', 'Cincinnati'],
    'OK': ['Oklahoma', 'Oklahoma City', 'Tulsa', 'Norman'],
    'OR': ['Oregon', 'Salem', 'Portland', 'Eugene'],
    'PA': ['Pennsylvania', 'Harrisburg', 'Philadelphia', 'Pittsburgh'],
    'RI': ['Rhode Island', 'Providence', 'Warwick', 'Cranston'],
    'SC': ['South Carolina', 'Columbia', 'Charleston', 'North Charleston'],
    'SD': ['South Dakota', 'Pierre', 'Sioux Falls', 'Rapid City'],
    'TN': ['Tennessee', 'Nashville', 'Memphis', 'Knoxville'],
    'TX': ['Texas', 'Austin', 'Houston', 'San Antonio'],
    'UT': ['Utah', 'Salt Lake City', 'West Valley City', 'Provo'],
    'VT': ['Vermont', 'Montpelier', 'Burlington', 'South Burlington'],
    'VA': ['Virginia', 'Richmond', 'Virginia Beach', 'Norfolk'],
    'WA': ['Washington', 'Olympia', 'Seattle', 'Spokane'],
    'WV': ['West Virginia', 'Charleston', 'Huntington', 'Morgantown'],
    'WI': ['Wisconsin', 'Madison', 'Milwaukee', 'Green Bay'],
    'WY': ['Wyoming', 'Cheyenne', 'Casper', 'Laramie']
}

import ahocorasick

A = ahocorasick.Automaton()
for abbr, names in states_dict.items():
    for name in names:
        A.add_word(name.lower(), (name, abbr))
A.make_automaton()

def fast_extract_state(text):
    text = str(text).lower()
    found_states = set()

    for _, (_, abbr) in A.iter(text):
        found_states.add(abbr)

    return list(found_states)

df_filtered = df_filtered.with_columns(
    pl.col("full_text").map_elements(fast_extract_state, return_dtype=pl.List(pl.Utf8)).alias("state")
)

df_filtered.head()

state,date,month_year,year,event_code,quad_class,goldstein_scale,avg_tone,actor1_statecode,actor2_statecode,url,title,full_text
list[str],i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str
"[""WA"", ""WI"", … ""MA""]",20210514,202105,2021,16,1,-2.0,-8.934073,"""USMO""","""USMO""","""https://www.natlawreview.com/a…","""State of the Law for Business …","""It’s been a year since COVID-1…"
"[""KS"", ""WA"", ""MO""]",20210514,202105,2021,141,3,-6.5,-0.808625,"""USMO""","""USMO""","""https://www.kcur.org/health/20…","""Medicaid Expansion Supporters …","""A day after Missouri Gov. Mike…"
"[""NY"", ""MO""]",20210529,202105,2021,13,1,0.4,-6.008584,"""USMO""","""USMO""","""https://www.dailystar.co.uk/ne…","""Elderly woman sucker-punched t…","""Elderly woman sucker-punched t…"
null,20200207,202002,2020,16,1,-2.0,-8.0,"""USAR""",null,"""https://www.houstonchronicle.c…",null,null
"[""NY"", ""DE"", … ""MA""]",20201206,202012,2020,70,2,7.0,0.088106,"""USNH""","""USNH""","""https://www.fosters.com/story/…","""Historically Speaking: Adventu…","""Historically Speaking: Adventu…"


In [ ]:
# df_filtered.write_parquet("data/FILTERED_DATAFRAME_WITH_STATES.parquet")
# df_filtered = pl.read_parquet("data/FILTERED_DATAFRAME_WITH_STATES.parquet")

## 2.2. Topic modeling

In [9]:
# create a clean texts series to deal with empty cells
texts_clean = df_filtered.select(
    pl.col('full_text').fill_null("").cast(str)
).to_series().to_list()

# chunk generator - to apply LDA incrementally, for memory optimization
def text_batch_generator(texts, batch_size):
    for i in range(0, len(texts), batch_size):
        yield texts[i:i + batch_size]

# initialize vectorizer and LDA
vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
vectorizer.fit(texts_clean)
lda = LatentDirichletAllocation(n_components=10,
                                learning_method='online',
                                batch_size=100,
                                random_state=42
                                )

# apply LDA by batches
for batch_texts in text_batch_generator(texts_clean, batch_size=1000):
    dtm_batch = vectorizer.transform(batch_texts)
    lda.partial_fit(dtm_batch)

# get topic distributions and add to the dataframe as new columns
dtm_full = vectorizer.transform(texts_clean)
topic_distributions = lda.transform(dtm_full)

df_with_topics = df_filtered.with_columns(
    [pl.Series(f"topic_{i}", topic_distributions[:, i]) for i in range(lda.n_components)]
)

df_with_topics.head()


state,date,month_year,year,event_code,quad_class,goldstein_scale,avg_tone,actor1_statecode,actor2_statecode,url,title,full_text,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
list[str],i64,i64,i64,i64,i64,f64,f64,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"[""WA"", ""WI"", … ""MA""]",20210514,202105,2021,16,1,-2.0,-8.934073,"""USMO""","""USMO""","""https://www.natlawreview.com/a…","""State of the Law for Business …","""It’s been a year since COVID-1…",0.024562,0.000097,0.228645,0.187616,0.033062,0.000097,0.350562,0.003926,0.161016,0.010416
"[""KS"", ""WA"", ""MO""]",20210514,202105,2021,141,3,-6.5,-0.808625,"""USMO""","""USMO""","""https://www.kcur.org/health/20…","""Medicaid Expansion Supporters …","""A day after Missouri Gov. Mike…",0.006544,0.098271,0.390895,0.132563,0.0219,0.152496,0.196633,0.000233,0.000233,0.000233
"[""NY"", ""MO""]",20210529,202105,2021,13,1,0.4,-6.008584,"""USMO""","""USMO""","""https://www.dailystar.co.uk/ne…","""Elderly woman sucker-punched t…","""Elderly woman sucker-punched t…",0.01934,0.000465,0.000465,0.000465,0.152191,0.123239,0.031558,0.000465,0.671346,0.000465
null,20200207,202002,2020,16,1,-2.0,-8.0,"""USAR""",null,"""https://www.houstonchronicle.c…",null,null,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
"[""NY"", ""DE"", … ""MA""]",20201206,202012,2020,70,2,7.0,0.088106,"""USNH""","""USNH""","""https://www.fosters.com/story/…","""Historically Speaking: Adventu…","""Historically Speaking: Adventu…",0.005939,0.028983,0.190054,0.000172,0.389295,0.047708,0.010148,0.000172,0.126847,0.20068


## 2.3. Aggregating the data by month and state

Below we aggregate the data by month and state, for harmony with our terrorism panel. We obtain both the mean and the variance. An article is assigned to a state if that state was mentioned in its text (as per the list obtained on step 2.1).

All the other columns get discarded. Other relevant columns have been processed in our *terrorism_processing.ipynb* and the text is not needed anymore. We also keep the state and period (year, month) columns as well as a joint column of state and period.

In [19]:
# select topic columns
topic_cols = [f'topic_{i}' for i in range(10)]

df_aggregated = (
    df_with_topics
    .filter(pl.col('title').is_not_null())
    .with_columns(
        (pl.col('date') // 100).alias('period')
    )
    .explode('state')
    .filter(pl.col('state').is_not_null())
    # do group_by and obtain mean and variance
    .group_by(['state', 'period'])
    .agg([
        *[pl.col(topic).mean().alias(f'{topic}_mean') for topic in topic_cols],
        *[pl.col(topic).var().alias(f'{topic}_var') for topic in topic_cols],
    ])
    .with_columns(
        (pl.col('state') + "_" + pl.col('period').cast(pl.Utf8)).alias('state_period')
    )
    .sort('state_period')
)

df_aggregated


state,period,topic_0_mean,topic_1_mean,topic_2_mean,topic_3_mean,topic_4_mean,topic_5_mean,topic_6_mean,topic_7_mean,topic_8_mean,topic_9_mean,topic_0_var,topic_1_var,topic_2_var,topic_3_var,topic_4_var,topic_5_var,topic_6_var,topic_7_var,topic_8_var,topic_9_var,state_period
str,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
"""AK""",202001,0.0062,0.176019,0.134228,0.046965,0.039173,0.165012,0.158361,0.000528,0.105847,0.167668,0.000382,0.08745,0.035572,0.014467,0.007535,0.019503,0.037005,0.000002,0.031942,0.043333,"""AK_202001"""
"""AK""",202002,0.00614,0.229535,0.132027,0.035967,0.044017,0.150766,0.151576,0.000576,0.108995,0.140403,0.000292,0.115541,0.038064,0.008395,0.009673,0.021,0.039013,0.000003,0.043537,0.03494,"""AK_202002"""
"""AK""",202003,0.009126,0.187728,0.129136,0.223781,0.028164,0.162057,0.056634,0.000504,0.060937,0.141933,0.000335,0.124263,0.028777,0.081757,0.00565,0.02526,0.010215,0.000002,0.024379,0.03561,"""AK_202003"""
"""AK""",202004,0.008067,0.137361,0.15453,0.211661,0.023576,0.184967,0.076624,0.00598,0.047435,0.1498,0.000327,0.084534,0.035404,0.049256,0.00363,0.022603,0.010807,0.00519,0.013221,0.037571,"""AK_202004"""
"""AK""",202005,0.014773,0.082678,0.156551,0.214141,0.033504,0.186337,0.096047,0.000531,0.065151,0.150287,0.001657,0.043889,0.036581,0.057899,0.004879,0.022767,0.018059,0.000002,0.017122,0.041323,"""AK_202005"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""WY""",202102,0.006397,0.155883,0.13424,0.065846,0.042889,0.192026,0.178719,0.012297,0.10177,0.109931,0.000282,0.039414,0.032426,0.033008,0.008681,0.016387,0.031606,0.011174,0.039751,0.021688,"""WY_202102"""
"""WY""",202103,0.010679,0.10315,0.173409,0.100278,0.035296,0.195375,0.145272,0.021006,0.093479,0.122054,0.00165,0.020322,0.029198,0.036147,0.007616,0.018053,0.01978,0.019312,0.030987,0.026692,"""WY_202103"""
"""WY""",202104,0.010255,0.075212,0.16735,0.10578,0.048858,0.196286,0.12088,0.012154,0.107896,0.155328,0.000507,0.020575,0.036652,0.046952,0.010197,0.017927,0.023781,0.01042,0.040783,0.03577,"""WY_202104"""


In [18]:
df_aggregated.write_parquet("data/AGGREGATED_DATAFRAME.parquet")